# Analysing Telco Company's Customer Churn Rate

- The goal of this project is to analyse the customer base of the given telco company and to predict the probability of churn.

- The dataset was created by IBM and contains information about services provided to 7043 customers in California in Q3. A more detailed description is available on [IBM's website](https://community.ibm.com/community/user/blogs/steven-macko/2019/07/11/telco-customer-churn-1113).


The original dataset is provided by IBM in XLSX format. For the purpose of a more realistic analysis environment, the data has been moved into a PostgreSQL database inside a Docker container. The data entries were uploaded using pdAdmin. The schema is described in `schema.sql`.

### Connecting to the database

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

user = 'postgres'
password = 'password'
host = '127.0.0.1'
port = 5431
database = 'telco-db'

def get_connection():
    return create_engine(
        url="postgresql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )

engine = get_connection()

## Database overview

In [2]:
pd.read_sql(
    text("""
        SELECT table_name AS "Table Name"
        FROM information_schema.tables
        WHERE table_schema NOT IN ('pg_catalog', 'information_schema')
    """), engine)

,Table Name
0,location
1,services
2,status
3,local_population
4,demographics


### `location` table

In [3]:
pd.read_sql(text("""
                 SELECT *
                 FROM location
                 LIMIT 5
                 """), engine)

,LOCATION_ID,CUSTOMER_ID,COUNT,COUNTRY,STATE,CITY,ZIP_CODE,LAT_LONG,LATITUDE,LONGITUDE
0,OXCZEW7397,8779-QRDMV,1,United States,California,Los Angeles,90022,"34.02381, -118.156582",34.023810,-118.156582
1,FCCECI8494,7495-OOKFY,1,United States,California,Los Angeles,90063,"34.044271, -118.185237",34.044271,-118.185237
2,HEHUQY7254,1658-BYGOY,1,United States,California,Los Angeles,90065,"34.108833, -118.229715",34.108833,-118.229715
3,WIUHRF2613,4598-XLKNJ,1,United States,California,Inglewood,90303,"33.936291, -118.332639",33.936291,-118.332639
4,CFEZBF4415,4846-WHAFZ,1,United States,California,Whittier,90602,"33.972119, -118.020188",33.972119,-118.020188


This table contains location details of customers that are all based in California. The primary key of this table is composite and made up of `LOCATION_ID` and `CUSTOMER_ID`.

### `services` table

In [4]:
pd.read_sql(text("""
                 SELECT *
                 FROM services
                 LIMIT 5
                 """), engine)

,SERVICE_ID,CUSTOMER_ID,COUNT,QUARTER,REFERRED_A_FRIEND,N_REFERRED,MONTHS_TENURE,OFFER,PHONE_SERVICE,AVG_MONTHLY_LONG_DIST_CHARGE,...,UNLIMITED_DATA,CONTRACT_TYPE,PAPERLESS_BILLING,PAYMENT_METHOD,MONTHLY_CHARGE,TOTAL_CHARGE,TOTAL_REFUNDS,TOTAL_EXTRA_DATA_CHARGES,TOTAL_LONG_DIST_CHARGES,TOTAL_REVENUE
0,IJKDQVSWH3522,8779-QRDMV,1,Q3,No,0,1,None,No,0.00,...,No,Month-to-Month,Yes,Bank Withdrawal,39.65,39.65,0.00,20.0,0.00,59.65
1,BFKMZJAIE2285,7495-OOKFY,1,Q3,Yes,1,8,Offer E,Yes,48.85,...,Yes,Month-to-Month,Yes,Credit Card,80.65,633.30,0.00,0.0,390.80,1024.10
2,EIMVJQBMT7187,1658-BYGOY,1,Q3,No,0,18,Offer D,Yes,11.33,...,Yes,Month-to-Month,Yes,Bank Withdrawal,95.45,1752.55,45.61,0.0,203.94,1910.88
3,EROZQXDUU4979,4598-XLKNJ,1,Q3,Yes,1,25,Offer C,Yes,19.76,...,Yes,Month-to-Month,Yes,Bank Withdrawal,98.50,2514.50,13.43,0.0,494.00,2995.07
4,GEEYSJUHY6991,4846-WHAFZ,1,Q3,Yes,1,37,Offer C,Yes,6.33,...,Yes,Month-to-Month,Yes,Bank Withdrawal,76.50,2868.15,0.00,0.0,234.21,3102.36


In [5]:
pd.read_sql(text("""
                 SELECT column_name AS "Column Name"
                 FROM information_schema.columns
                 WHERE table_catalog = 'telco-db' AND table_name = 'services'
                 ORDER BY ordinal_position
                 """), engine)

,Column Name
0,SERVICE_ID
1,CUSTOMER_ID
2,COUNT
3,QUARTER
4,REFERRED_A_FRIEND
5,N_REFERRED
6,MONTHS_TENURE
7,OFFER
8,PHONE_SERVICE
9,AVG_MONTHLY_LONG_DIST_CHARGE


This table contains client-specific information about the service provided and the main information that will be useful when modelling churn.

### `status` table

In [6]:
pd.read_sql(text("""
                 SELECT *
                 FROM status
                 LIMIT 5
                 """), engine)

,STATUS_ID,CUSTOMER_ID,COUNT,QUARTER,SATISFACTION_SCORE,CUSTOMER_STATUS,CHURN_LABEL,CHURN_VALUE,CHURCH_SCORE,CLTV,CHURN_CATEGORY,CHURN_REASON
0,SWSORB1252,8779-QRDMV,1,Q3,3,Churned,Yes,True,91,5433,Competitor,Competitor offered more data
1,SNAEQA8572,7495-OOKFY,1,Q3,3,Churned,Yes,True,69,5302,Competitor,Competitor made better offer
2,LMBQNN3714,1658-BYGOY,1,Q3,2,Churned,Yes,True,81,3179,Competitor,Competitor made better offer
3,VRZYZI9978,4598-XLKNJ,1,Q3,2,Churned,Yes,True,88,5337,Dissatisfaction,Limited range of services
4,FDNAKX1688,4846-WHAFZ,1,Q3,2,Churned,Yes,True,67,2793,Price,Extra data charges


This table shows information regarding the currect customer's satisfaction and whether they have or have not churned. Information about the cause of churn is also provided along with customer lifetime value which can also be modelled.

### `local_population` table

In [7]:
pd.read_sql(text("""
                 SELECT *
                 FROM local_population
                 LIMIT 5
                 """), engine)

,ID,ZIP_CODE,POPULATION
0,1,90001,54492
1,2,90002,44586
2,3,90003,58198
3,4,90004,67852
4,5,90005,43019


This table shows the local population in a given zip code. This can be used to discert more urban areas from more rural ones.

### `demographics` table

In [8]:
pd.read_sql(text("""
                 SELECT *
                 FROM demographics
                 LIMIT 5
                 """), engine)

,CUSTOMER_ID,COUNT,GENDER,AGE,UNDER_30,IS_SENIOR,IS_MARRIED,HAS_DEPENDENTS,N_DEPENDENTS
0,8779-QRDMV,1,Male,78,No,Yes,No,No,0
1,7495-OOKFY,1,Female,74,No,Yes,Yes,Yes,1
2,1658-BYGOY,1,Male,71,No,Yes,No,Yes,3
3,4598-XLKNJ,1,Female,78,No,Yes,Yes,Yes,1
4,4846-WHAFZ,1,Female,80,No,Yes,Yes,Yes,1


This table describes the person's demographic details such as age, gender and family status.

## Exploring the data

One way to analyse this data would be merging all tables and performing analysys on Pandas's DataFrame but let's instead perform basic analysis through SQL itself.